# Парсинг данных с auto.ru

In [4]:
import os
import time
import datetime

import requests as r
import json
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

In [6]:
response = r.get("https://auto.ru/moskva/cars/used/")
response.encoding = 'utf-8'

In [7]:
page = BeautifulSoup(response.text, 'html.parser')

In [8]:
json_data = json.loads(page.find('script', id='initial-state').string)

In [9]:
marks = list(map(lambda x: x['itemFilterParams']['mark'], json_data['breadcrumbsPublicApi']['data'][0]['entities']))

In [48]:
marks

['AC',
 'ACURA',
 'ADLER',
 'ALFA_ROMEO',
 'ALPINA',
 'ALPINE',
 'AM_GENERAL',
 'AMC',
 'APAL',
 'ARIEL',
 'ARO',
 'ASIA',
 'ASTON_MARTIN',
 'AUBURN',
 'AUDI',
 'AURUS',
 'AUSTIN',
 'AUSTIN_HEALEY',
 'AUTOBIANCHI',
 'BAIC',
 'BAJAJ',
 'BALTIJAS_DZIPS',
 'BATMOBILE',
 'BENTLEY',
 'BERTONE',
 'BILENKIN',
 'BIO_AUTO',
 'BITTER',
 'BMW',
 'BORGWARD',
 'BRABUS',
 'BRILLIANCE',
 'BRISTOL',
 'BUFORI',
 'BUGATTI',
 'BUICK',
 'BYD',
 'BYVIN',
 'CADILLAC',
 'CALLAWAY',
 'CARBODIES',
 'CATERHAM',
 'CHANA',
 'CHANGAN',
 'CHANGFENG',
 'CHANGHE',
 'CHERY',
 'CHERYEXEED',
 'CHEVROLET',
 'CHRYSLER',
 'CITROEN',
 'CIZETA',
 'COGGIOLA',
 'CORD',
 'CUPRA',
 'DACIA',
 'DADI',
 'DAEWOO',
 'DAIHATSU',
 'DAIMLER',
 'DALLARA',
 'DATSUN',
 'DE_TOMASO',
 'DECO_RIDES',
 'DELAGE',
 'DELOREAN',
 'DERWAYS',
 'DESOTO',
 'DKW',
 'DODGE',
 'DONGFENG',
 'DONINVEST',
 'DONKERVOORT',
 'DS',
 'DW_HOWER',
 'E_CAR',
 'EAGLE',
 'EAGLE_CARS',
 'EXCALIBUR',
 'FAW',
 'FERRARI',
 'FIAT',
 'FISKER',
 'FLANKER',
 'FORD',
 'FOTON',

In [11]:
def get_params_and_headers(mark: str, page_num: int, transmission: str):
    PARAMS = {
         'catalog_filter' : [{"mark": mark}],
         'section': 'used',
         'category': 'cars',
         'sort': 'fresh_relevance_1-desc',
         'page': page_num
        }
    
    if len(transmission) > 0:
        PARAMS['transmission'] = transmission
    
    HEADERS = {
        'Accept': '*/*',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
        'Connection': 'keep-alive',
        'content-type': 'application/json',
        'Cookie': '_csrf_token=12d20142d10520f7b82c3dec0761a49668ff01a261b9171c; autoru_sid=a:g608d8c462ardieoc0p4gq330989vaks.c732a85395b1c9ec165133ee4095e6a6|1619889222514.604800.HkPE-x1SxJKvhQahdUKJSA.VSWxbotagLLIQcVL5CB-znX8JlN0JrRgEnPJcWp-cZA; autoruuid=g608d8c462ardieoc0p4gq330989vaks.c732a85395b1c9ec165133ee4095e6a6; autoru_gdpr=1; suid=82ade37eba6f1ecf225433ca6047bb66.047145ec6efb2e2ae89d4d21754e2986; from=direct; yuidlt=1; yandexuid=8117481671618082737; my=YwA=; crookie=PrzkiqRgqdN/tIYCniScgxCemcKp24LXjs81XXozOneCz4wUfylD6HFpbUGEjxb7zS1A5gUXNT3f5DG+MozPfSzHHtE=; cmtchd=MTYxOTg4OTIzNzk5OA==; gdpr=0; _ym_isad=1; _ym_uid=161988924290763830; autoru-visits-count=1; X-Vertis-DC=sas; gids=213; bltsr=1; gradius=0; from_lifetime=1619899678001; _ym_d=1619900682',
        'Host': 'auto.ru',
        'origin': 'https://auto.ru',
        'Referer': f'https://auto.ru/moskva/cars/{mark.lower()}/used/',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0',
        'x-client-app-version': '202002.03.092255',
        'x-client-date': '1580933207763',
        'x-csrf-token': '12d20142d10520f7b82c3dec0761a49668ff01a261b9171c',
        'x-page-request-id': '60142cd4f0c0edf51f96fd0134c6f02a',
        'x-requested-with': 'fetch'
    }
    
    return PARAMS, HEADERS

In [61]:
URL = 'https://auto.ru/-/ajax/desktop/listing/'

In [62]:
PATH = '/.cars/'

In [14]:
transmissions = ['AUTOMATIC', 'ROBOT', 'VARIATOR', 'MECHANICAL']

In [63]:
# Сохраним найденную информацию в файловую систему

for mark in marks:
    for transmission in transmissions:
        print(mark.lower() + '-' + transmission.lower(), end=': ')

        new_path = PATH + mark.lower() + '-' + transmission.lower()

        if not os.path.exists(new_path):
            os.makedirs(new_path)
        
        # goes by pages from 1 to 100
        for i in range(1, 100):        
            PARAMS, HEADERS = get_params_and_headers(mark, i, transmission)    
            response = requests.post(URL, json=PARAMS, headers=HEADERS)
            response.encoding = 'utf-8'
            offers = response.json()['offers']

            cars_count = len(offers)

            if cars_count == 0:
                break;

            new_file_path = new_path + '/page-' + str(i) + '.txt'

            if not os.path.exists(new_file_path):
                with open(new_file_path, 'w') as out_file:
                    out_file.write(json.dumps(offers))

            print(i, f'({cars_count})', end='  ')

        print()

ac-automatic: 
ac-robot: 
ac-variator: 
ac-mechanical: 1 (2)  
acura-automatic: 1 (37)  2 (2)  
acura-robot: 1 (3)  
acura-variator: 
acura-mechanical: 
adler-automatic: 
adler-robot: 
adler-variator: 
adler-mechanical: 
alfa_romeo-automatic: 1 (7)  
alfa_romeo-robot: 1 (16)  
alfa_romeo-variator: 
alfa_romeo-mechanical: 1 (23)  
alpina-automatic: 1 (6)  
alpina-robot: 
alpina-variator: 
alpina-mechanical: 
alpine-automatic: 
alpine-robot: 
alpine-variator: 
alpine-mechanical: 
am_general-automatic: 
am_general-robot: 
am_general-variator: 
am_general-mechanical: 
amc-automatic: 1 (1)  
amc-robot: 
amc-variator: 
amc-mechanical: 1 (2)  
apal-automatic: 
apal-robot: 
apal-variator: 
apal-mechanical: 
ariel-automatic: 
ariel-robot: 
ariel-variator: 
ariel-mechanical: 1 (1)  
aro-automatic: 
aro-robot: 
aro-variator: 
aro-mechanical: 1 (1)  
asia-automatic: 
asia-robot: 
asia-variator: 
asia-mechanical: 
aston_martin-automatic: 1 (3)  
aston_martin-robot: 
aston_martin-variator: 
aston_ma

In [64]:
test_df = pd.read_csv('data/test.csv', header=0)

In [65]:
t = time.gmtime()

In [66]:
colors_dict = {
    '040001' : 'чёрный',
    'FAFBFB': 'белый',
    '97948F': 'серебристый',
    'CACECB': 'серый',
    '0000CC': 'синий',
    'EE1D19': 'красный',
    '200204': 'коричневый',
    '007F00': 'зелёный',
    'C49648': 'бежевый',
    '22A0F8': 'голубой',
    'FFD600': 'золотистый',
    '660099': 'пурпурный',
    '4A2197': 'фиолетовый',
    'DEA522': 'жёлтый',
    'FF8649': 'оранжевый',
    'FFC0CB': 'розовый'
}

In [67]:
steering_wheel_dict = {
    'LEFT': 'Левый',
    'RIGHT': 'Правый'
}

In [68]:
pts_dict = {
    'ORIGINAL': 'Оригинал',
    'DUPLICATE': 'Дубликат'
}

In [69]:
fuel_dict = {
    'GASOLINE': 'бензин',
    'DIESEL': 'дизель',
    'HYBRID': 'гибрид',
    'ELECTRO': 'электро',
    'LPG': 'газ'
}

In [70]:
gear_type_dict = {
    'FORWARD_CONTROL': 'передний',
    'ALL_WHEEL_DRIVE': 'полный',
    'REAR_DRIVE': 'задний'
}

In [71]:
transmission_dict = {
    'AUTOMATIC': 'автоматическая',
    'MECHANICAL': 'механическая',
    'ROBOT': 'роботизированная',
    'VARIATOR': 'вариатор'
}

In [72]:
def get_owner_count(x):
    if x == 1:
        return '1\xa0владелец'
    elif x == 2:
        return '2\xa0владельца'
    elif x > 2:
        return '3 или более'
    
    return None

In [73]:
def get_year_ru(x):
    if x % 10 == 1 and x != 11 and x % 100 != 11:
        return  'год'
    elif 1 < x % 10 <= 4 and x != 12 and x != 13 and x != 14:
        return 'года'
    
    return 'лет'

In [74]:
def get_month_ru(m):
    if m == 1:
        return 'месяц'
    elif m == 2 or m == 3 or m == 4:
        return 'месяца'
    return 'месяцев'

In [75]:
def get_ownership_period(months):
    if months == 0 or months == None:
        return None
    
    if months > 11:
        years = months // 12
        months = months % 12
        if months > 0:
            return f'{years} {get_year_ru(years)} и {months} {get_month_ru(months)}'
        else:
            return f'{years} {get_year_ru(years)}'
    else:
        return f'{months} {get_month_ru(months)}'

In [76]:
cars_list = []

# read cars from file system
for f in os.scandir(PATH):
    for file in os.scandir(f.path):
        with open(file.path, 'r') as r:
            page_json = json.loads(r.read())
            for car in page_json:
                model_info = car['vehicle_info']['model_info']
                
                brand = car['vehicle_info']['mark_info']['code']
                model_name = model_info['code']
                saleId = car['saleId']
                car_url = f'https://auto.ru/cars/used/sale/{brand.lower()}/{model_name.lower()}/{saleId}/'
                color = colors_dict[car['color_hex']]
                
                complectation_dict = car['vehicle_info']['complectation']
                if complectation_dict['id'] == "0":
                    complectation_dict = None
                    
                bodyType = car['vehicle_info']['configuration']['human_name'].lower()
                try: description = car['description']
                except: description = None
                engineDisplacement = str(round(int(car['vehicle_info']['tech_param']['displacement'])/1000, 1)) + ' LTR'
                enginePower = str(car['vehicle_info']['tech_param']['power']) + ' N12'
                equipment_dict = car['vehicle_info']['equipment']
                fuelType = fuel_dict[car['vehicle_info']['tech_param']['engine_type']]
                # image = car[]
                mileage = car['state']['mileage']
                modelDate = car['vehicle_info']['super_gen']['year_from']
            
                name = car['vehicle_info']['tech_param']['human_name']
                numberOfDoors = car['vehicle_info']['configuration']['doors_count']
                parsing_unixtime = round(time.time())
                
                if 'currency' in car['price_info']:
                    priceCurrency = car['price_info']['currency']
                    if priceCurrency == 'RUR':
                        priceCurrency = 'RUB'
                else:
                    priceCurrency = None
                
                if 'price' in car['price_info']:
                    price = car['price_info']['price']
                else:
                    price = None
                    
                productionDate = car['documents']['year']
                sell_id = car['id']
                super_gen = car['vehicle_info']['tech_param']
                vehicleConfiguration = f"{(car['vehicle_info']['configuration']['body_type'])} {(car['vehicle_info']['tech_param']['transmission'])} {engineDisplacement}"
                vehicleTransmission = transmission_dict[car['vehicle_info']['tech_param']['transmission']]
                vendor = car['vehicle_info']['vendor']
                
                if 'owners_number' in car['documents']:
                    owners_number = get_owner_count(car['documents']['owners_number'])
                else:
                    owners_number = None
                
                if 'purchase_date' in car['documents']:
                    dt_now = datetime.datetime(t.tm_year, t.tm_mon, 1)
                    td_purchase = datetime.datetime(car['documents']['purchase_date']['year'], car['documents']['purchase_date']['month'], 1)
                    num_months = (dt_now.year - td_purchase.year) * 12 + (dt_now.month - td_purchase.month) + 1
                    ownership_period = get_ownership_period(num_months)
                else:
                    ownership_period = None
                
                if 'pts' in car['documents']:
                    pts = pts_dict[car['documents']['pts']]
                else:
                    pts = None
                
                gear_type = gear_type_dict[car['vehicle_info']['tech_param']['gear_type']]
                steering_wheel = steering_wheel_dict[car['vehicle_info']['steering_wheel']]
                state = 'Не требует ремонта' if car['state']['state_not_beaten'] == True else 'Требует ремонта'
                custom_cleared = 'Растаможен' if car['documents']['custom_cleared'] == True else 'Не растаможен'
                
                cars_list.append({
                    'brand': brand, 
                    'saleId': saleId, 
                    'car_url': car_url, 
                    'color': color,
                    'complectation_dict': complectation_dict, 
                    'bodyType': bodyType, 
                    'description': description, 
                    'engineDisplacement': engineDisplacement, 
                    'enginePower': enginePower, 
                    'equipment_dict': equipment_dict, 
                    'fuelType': fuelType, 
                    'image': None, 
                    'mileage': mileage, 
                    'modelDate': modelDate, 
                    'model_info': model_info, 
                    'model_name': model_name, 
                    'name': name, 
                    'numberOfDoors': numberOfDoors, 
                    'parsing_unixtime': parsing_unixtime, 
                    'priceCurrency': priceCurrency, 
                    'productionDate': productionDate, 
                    'sell_id': sell_id, 
                    'super_gen': super_gen, 
                    'vehicleConfiguration': vehicleConfiguration, 
                    'vehicleTransmission': vehicleTransmission, 
                    'vendor': vendor, 
                    'Владельцы': owners_number, 
                    'Владение': ownership_period, 
                    'ПТС': pts, 
                    'Привод': gear_type, 
                    'Руль': steering_wheel, 
                    'Состояние': state, 
                    'Таможня': custom_cleared,
                    'price': price
                })

In [77]:
len(cars_list)

53778

In [78]:
cars_df = pd.DataFrame(cars_list, columns=[
    'bodyType',
    'brand', 
    'car_url', 
    'color',
    'complectation_dict',
    'description', 
    'engineDisplacement', 
    'enginePower', 
    'equipment_dict',
    'fuelType', 
    'image', 
    'mileage', 
    'modelDate', 
    'model_info', 
    'model_name',
    'name', 
    'numberOfDoors', 
    'parsing_unixtime', 
    'priceCurrency',
    'productionDate', 
    'sell_id', 
    'super_gen', 
    'vehicleConfiguration',
    'vehicleTransmission', 
    'vendor',
    'Владельцы',
    'Владение',
    'ПТС',
    'Привод',
    'Руль',
    'Состояние',
    'Таможня',
    'price'
])

In [79]:
# удаляем записи где не указано количество владельцев (их мало, всего 1)
cars_df = cars_df[~cars_df['Владельцы'].isna()]

In [80]:
# удаляем дубликаты
cars_df = cars_df.drop_duplicates(subset=['sell_id'])

In [81]:
# удаляем записи где нет цены
cars_df = cars_df[~cars_df['price'].isna()]

In [82]:
cars_df.to_csv('data/parsed_data.csv', index=False)

In [7]:
pd.reset_option('display.max_colwidth')

In [10]:
data_mine = pd.read_csv('data/parsed_data.csv')
data_mine.sample(5)

,bodyType,brand,car_url,color,complectation_dict,description,engineDisplacement,enginePower,equipment_dict,fuelType,image,mileage,modelDate,model_info,model_name,name,numberOfDoors,parsing_unixtime,priceCurrency,productionDate,sell_id,super_gen,vehicleConfiguration,vehicleTransmission,vendor,Владельцы,Владение,ПТС,Привод,Руль,Состояние,Таможня,price
42881,минивэн lucida,TOYOTA,https://auto.ru/cars/used/sale/toyota/estima/1...,серебристый,NaN,"Тойота эстима задний привод, очень долго стоял...",2.2 LTR,100 N12,{},дизель,NaN,180000,1990,"{'code': 'ESTIMA', 'name': 'Estima', 'ru_name'...",ESTIMA,Lucida 2.2d AT (100 л.с.),4,1623674092,RUB,1992,1103802124,"{'id': '8305898', 'name': 'Lucida', 'nameplate...",MINIVAN AUTOMATIC 2.2 LTR,автоматическая,JAPANESE,2 владельца,NaN,Оригинал,задний,Правый,Не требует ремонта,Растаможен,245000.0
51938,внедорожник 5 дв.,VOLVO,https://auto.ru/cars/used/sale/volvo/xc90/1102...,чёрный,NaN,Фото с зимы свежих пока нету но машина такая ж...,2.5 LTR,210 N12,{},бензин,NaN,194000,2002,"{'code': 'XC90', 'name': 'XC90', 'ru_name': 'X...",XC90,2.5 AT (210 л.с.) 4WD,5,1623674095,RUB,2004,1102429461,"{'id': '6367800', 'displacement': 2521, 'engin...",ALLROAD_5_DOORS AUTOMATIC 2.5 LTR,автоматическая,EUROPEAN,3 или более,NaN,Оригинал,полный,Левый,Не требует ремонта,Растаможен,520000.0
16320,седан,HYUNDAI,https://auto.ru/cars/used/sale/hyundai/accent/...,серебристый,NaN,Специальные программы кредитования:\n▶ Оформле...,1.5 LTR,92 N12,"{'alloy-wheel-disks': True, 'condition': True,...",бензин,NaN,177591,1999,"{'code': 'ACCENT', 'name': 'Accent', 'ru_name'...",ACCENT,ТагАЗ 1.5 MT (92 л.с.),4,1623674081,RUB,2004,1103499845,"{'id': '20392633', 'name': 'ТагАЗ', 'nameplate...",SEDAN MECHANICAL 1.5 LTR,механическая,KOREAN,3 или более,NaN,Оригинал,передний,Левый,Не требует ремонта,Растаможен,162800.0
20788,седан,KIA,https://auto.ru/cars/used/sale/kia/quoris/1103...,белый,NaN,ЛОТ: 01264311\nFAVORIT MOTORS KIA МКАД\n\nВы м...,3.8 LTR,334 N12,"{'cruise-control': True, 'asr': True, 'tinted-...",бензин,NaN,109486,2014,"{'code': 'QUORIS', 'name': 'Quoris', 'ru_name'...",QUORIS,3.8 AT (334 л.с.),4,1623674083,RUB,2014,1103564513,"{'id': '20229135', 'displacement': 3778, 'engi...",SEDAN AUTOMATIC 3.8 LTR,автоматическая,KOREAN,3 или более,NaN,Оригинал,задний,Левый,Не требует ремонта,Растаможен,1440000.0
40433,хэтчбек 3 дв.,SMART,https://auto.ru/cars/used/sale/smart/fortwo/11...,красный,NaN,Отличное состояние. Без ДТП. Турбо мотор. Комп...,1.0 LTR,84 N12,"{'engine-proof': True, 'cruise-control': True,...",бензин,NaN,27500,2007,"{'code': 'FORTWO', 'name': 'Fortwo', 'ru_name'...",FORTWO,1.0 AMT (84 л.с.),3,1623674091,RUB,2015,1103552615,"{'id': '8349492', 'displacement': 999, 'engine...",HATCHBACK_3_DOORS ROBOT 1.0 LTR,роботизированная,EUROPEAN,3 или более,8 месяцев,Оригинал,задний,Левый,Не требует ремонта,Растаможен,725000.0


In [11]:
data_mine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53019 entries, 0 to 53018
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bodyType              53019 non-null  object 
 1   brand                 53019 non-null  object 
 2   car_url               53019 non-null  object 
 3   color                 53019 non-null  object 
 4   complectation_dict    10081 non-null  object 
 5   description           51277 non-null  object 
 6   engineDisplacement    53019 non-null  object 
 7   enginePower           53019 non-null  object 
 8   equipment_dict        53019 non-null  object 
 9   fuelType              53019 non-null  object 
 10  image                 0 non-null      float64
 11  mileage               53019 non-null  int64  
 12  modelDate             53019 non-null  int64  
 13  model_info            53019 non-null  object 
 14  model_name            53019 non-null  object 
 15  name               